In [1]:
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
infos = pd.read_excel("../../1_NanoNASCseq/NanoNASCseq_summary_selected_qc.xls")

# Make counts for Seurat

In [2]:
dat = infos[(infos["Strain"] == "K562") & (np.isnan(infos["ActD"]))]
print("Cells:", len(dat))

array = []
for run, cell in dat[["Run", "Cell"]].values:
    path = "../../1_NanoNASCseq/results/expression/quantify/2TC/%s/%s/quant_gene.tsv.gz" % (run, cell)
    d = pd.read_csv(gzip.open(path), sep="\t", index_col=0)
    s = d["Total"]
    s.name = cell
    array.append(s)

Cells: 446


In [3]:
m = pd.concat(array, axis=1).fillna(0)

anno_g = pd.read_csv("/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv", sep="\t")
anno_g.index = anno_g["GeneID"]

gid2gname = {gid: gname for gid, gname in anno_g[["GeneID", "GeneName"]].values}
gids = set(anno_g["GeneID"])

m = m[[gid in gids for gid in m.index]]
m.index = [gid2gname[gid] for gid in m.index]
m = m[~m.index.duplicated()]
m.index.name = "GeneName"

m.to_csv("data/K562_counts.tsv", sep="\t")
dat.to_csv("data/K562_meta.tsv", sep="\t", index=False)